In [ ]:
import os
import json
import pickle
import numpy as np
from pyvi import ViTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn import metrics

# Hàm tiền xử lý dữ liệu
def preprocess(data_path, stopword_path, dims):
    data = []
    target = []
    target_names = []

    # Duyệt qua từng file trong thư mục
    for file_name in os.listdir(data_path):
        if file_name.endswith(".json"):  # Chỉ xử lý file .json
            file_path = os.path.join(data_path, file_name)
            category = os.path.splitext(file_name)[0]
            if category not in target_names:
                target_names.append(category)

            with open(file_path, "r", encoding="utf-8") as file:
                json_data = json.load(file)
                for item in json_data:
                    content = item.get("content", "").strip()
                    if content.lower() == "none":
                        continue
                    text = (item.get("title", "") + " " + item.get("content", "")).strip()
                    data.append(text)
                    target.append(target_names.index(category))
                    
    # Đọc danh sách từ dừng
    with open(stopword_path, encoding="utf-8") as f:
        stopwords = [x.strip().replace(" ", "_") for x in f.readlines()]

    # Tách từ tiếng Việt
    data = [ViTokenizer.tokenize(sample) for sample in data]

    # Chuyển văn bản thành biểu diễn TF-IDF vector
    tfidf_vector = TfidfVectorizer(stop_words=stopwords)
    data_preprocessed = tfidf_vector.fit_transform(data)

    # Lưu từ điển TF-IDF
    with open("tfidf_vocabulary.pkl", "wb") as f:
        pickle.dump(tfidf_vector.vocabulary_, f)

    # Chuyển dữ liệu TF-IDF thành ma trận
    tfidf_mat = data_preprocessed.toarray()

    # Áp dụng PCA để giảm chiều dữ liệu
    pca = PCA(n_components=dims)
    x_reduced = pca.fit_transform(tfidf_mat)

    # Lưu PCA đã huấn luyện
    with open("pca_model.pkl", "wb") as f:
        pickle.dump(pca, f)

    return x_reduced, np.array(target), target_names

# Hàm huấn luyện mô hình
def train_model(classifier, X_data, y_data):
    X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)
    classifier.fit(X_train, y_train)

    # Đánh giá mô hình
    train_accuracy = metrics.accuracy_score(y_train, classifier.predict(X_train))
    test_accuracy = metrics.accuracy_score(y_test, classifier.predict(X_test))
    print(f"Train accuracy: {train_accuracy:.2f}")
    print(f"Test accuracy: {test_accuracy:.2f}")

# Hàm dự đoán
def predict_text(classifier, text, stopwords):
    # Tải từ điển TF-IDF đã lưu
    with open("tfidf_vocabulary.pkl", "rb") as f:
        vocabulary = pickle.load(f)

    # Khởi tạo TfidfVectorizer với từ điển đã học
    tfidf_vectorizer = TfidfVectorizer(stop_words=stopwords, vocabulary=vocabulary)

    # Tách từ tiếng Việt
    tokenized_text = ViTokenizer.tokenize(text)
    print("Câu sau khi tách từ:", tokenized_text)

    # Chuyển văn bản thành vector TF-IDF
    tfidf_vector = tfidf_vectorizer.transform([tokenized_text])
    tfidf_mat = tfidf_vector.toarray()

    # Tải PCA đã lưu
    with open("pca_model.pkl", "rb") as f:
        pca = pickle.load(f)

    # Giảm chiều dữ liệu bằng PCA
    tfidf_reduced = pca.transform(tfidf_mat)

    # Dự đoán nhãn
    prediction = classifier.predict(tfidf_reduced)
    return prediction

# Đường dẫn tới dữ liệu
data_path = "json"  # Thư mục chứa các file JSON
stopword_path = "vietnamese-stopwords.txt"
dims = 1024

# Tiền xử lý dữ liệu
X_data, y_data, target_names = preprocess(data_path, stopword_path, dims)
print(f"Dữ liệu đã tiền xử lý: {X_data.shape}")
print(f"Số nhãn: {len(target_names)}")
print("Danh sách nhãn và ID tương ứng:")
for idx, name in enumerate(target_names):
    print(f"ID: {idx}, Nhãn: {name}")

# Huấn luyện mô hình
naive_bayes_classifier = GaussianNB()
train_model(naive_bayes_classifier, X_data, y_data)

# Văn bản cần dự đoán
input_text = "Ca sĩ Hiếu Thứ Hai ra MV mới"

# Dự đoán văn bản
with open(stopword_path, "r", encoding="utf-8") as f:
    stopwords = [line.strip() for line in f]

predicted_label = predict_text(naive_bayes_classifier, input_text, stopwords)

# Hiển thị kết quả
print(f"Dự đoán nhãn: {target_names[predicted_label[0]]}")
